In [13]:
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [23]:
tweets = pd.read_csv("../../tweets.csv", index_col=None)

In [26]:
english_tweets = tweets[tweets["lang"] == "en"]
len(english_tweets)

12998

In [46]:
user_tweets = list(zip(english_tweets["screen_name"].values, english_tweets["full_text"].values))
user_sentences = []
sententces = []
for idx, (uid, tweet) in enumerate(user_tweets):
    for s in sent_tokenize(tweet):
        user_sentences.append((idx, uid, s))
        sentences.append(s)

In [15]:
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

In [48]:
    model.encode(sentences)

KeyboardInterrupt: 